# Introduction

<p>The dataset contains the U.S. border crossing, which contains information of the inbound crossings at the U.S.-Canada ana U.S.- Mexico borders.

COLUMNS DESCRIPTION  <br/>
> Port Name: Name of the Port of Entry <br/>
> State: State <br/>
> Port Code: CBP port code <br/>
> Border: US-Canada Border [](http://)or US-Mexico Border <br/>
> Date: Year, Month <br/>
> Measure: Conveyances,Means <br/>
> Value: Count <br/>
> Location: Longitude and Latitude Location

___

importing of relevant libraries 

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
sns.set(palette=("Set1"))

In [ ]:
df = pd.read_csv("../input/border-crossing-entry-data/Border_Crossing_Entry_Data.csv")
df

# Exploration and Visualization

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

There is no missing value in the data 

Using Pandas Describe object for futher Analysis

In [ ]:
df.describe(include='all')

Checking for duplicated rows

In [ ]:
df.duplicated().sum()

Since the Port code column is an idientification value for Port Name I have to check if it has the same unique values as the Port Name column 

In [ ]:
print('Unique values in Port Name: '+str(df['Port Name'].nunique() ))
print("Unique values in Port Code: " +str(df['Port Code'].nunique()))

They are not the same!!
<p> 

Since they are not the same, I have to fix the problem

In [ ]:
port=df[['Port Name','Port Code']].drop_duplicates()
port[port['Port Name'].duplicated(keep = False)]

In [ ]:
df.loc[(df['Port Code']==103) | (df['Port Code']==3302)]


From the code above I can see that Eastport have to different port code and the two Eastport are located at different states
<p>I now have to rename it to avoid further confusion 


Rename Eastport located in Maine to 'Eastport Maine' to avoid confusion and bais

In [ ]:
df.loc[(df['Port Name'] == 'Eastport') & (df['State'] == 'Maine'), 'Port Name'] = 'Eastport Maine'

Converting Date column to pandas datetime

In [ ]:

Date = pd.to_datetime(df['Date'])
Date

In [ ]:
df2 = pd.DataFrame(df)
df2

Creating Year and Month Column from Date Column

In [ ]:
df2['Year'] =Date.dt.year
df2['Month'] =Date.dt.month
df2

Creating Longitude and Latitude from Location Column

In [ ]:
def longitude(x):
    b1= x.split('(')[1]
    b2 = b1.split(')')
    for i in b2:
         return i.split()[0]

def latitude(x):
    b1= x.split('(')[1]
    b2 = b1.split(')')
    for i in b2:
         return i.split()[1]

df2['latitude'] = df2.Location.apply(latitude)
df2['longitude'] = df2.Location.apply(longitude)


In [ ]:
df2

Which border has the most Occurrence? 

In [ ]:
Border_count = df2.Border.value_counts()
Border_count

In [ ]:
plt.figure(figsize=(7,6))
plt.pie(x = Border_count.values,explode=[0.03,0.03], labels = Border_count.index, autopct='%0.2f%%',)
plt.title('Composition of the Borders')
plt.show()



From the chart we can see that US-Canada Border 76.77% has more occurrence than US-Mexico Border 23.23%

Which states have the highest Occurrence in each border ?


Using the pandas group function. I have the table below:

In [ ]:
df2.groupby(['Border','State']).count()

From the table above, it can been seen that North Dakota has the highest occurrence in US-Canada Border while Texas has the highest occurrence in US-Mexico Border
<p> Visualizing it, we have the bar chart below.

In [ ]:

g1 = sns.catplot('State',col="Border",data=df2, kind='count', height=8, aspect=1,col_wrap=3,)
g1.set_ylabels(fontsize=15)
g1.set_xlabels(fontsize=15)
g1.set_xticklabels(rotation=80,fontsize=13)
plt.show()

which Ports have the highest occurrence?

In the output of pandas describe object, it was show that port has 117 unique values, which is 117 ports. 
<p>Visualizing ports using word cloud and matplotlib we have the following below


In [ ]:
df2['Port Name'].value_counts()

In [ ]:
from wordcloud import WordCloud, ImageColorGenerator
text = " ".join(str(each) for each in df2['Port Name'])
wordcloud = WordCloud(max_words=100, background_color="White").generate(text)
plt.figure(figsize=(15,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title('Most Ports used')
plt.show()

From the image above the Ports which its name appear in big fonts have the highest occurrences

Which Measures have the hieght occurrences ?

In [ ]:
Measure_count =df2.Measure.value_counts()

In [ ]:
plt.figure(figsize=(8,7))
plt.pie(x = Measure_count.values,labels = Measure_count.index,autopct='%0.2f%%',pctdistance=0.8)
plt.title('Composition of the Measure')

plt.show()


From the above chart it can be shown that Trucks has the highest occurrences but its difference of from other measures is not much.

In [ ]:
Measure_group = df2.groupby(['Border','Measure']).count()
Measure_group

Visualizing with respect to the Border we have the chart below

In [ ]:
g2 = sns.catplot('Measure',col="Border",data=df2,kind='count', height=5.5, aspect=1,col_wrap=3)
g2.set_ylabels(fontsize=15)
g2.set_xlabels(fontsize=15)
g2.set_xticklabels(rotation=80,fontsize=13)
plt.show()

Which Year hass the most occurrences?

In [ ]:
year_count=df2.Year.value_counts()
year_count

In [ ]:
plt.figure(figsize = (12,7))
sns.lineplot(x= year_count.index,y=year_count.values)
plt.title('Distribution of Year',fontsize=18,)
plt.xlabel('Year',fontsize=18)
plt.show()

In the graph above 2010 and 2005 have the highest occurrences  and 2019 and 2018 have the lowest occurrences  in the data 

Visualizing Year with respect to measure

In [ ]:
measure_year =df2.groupby(['Border','Year','Measure']).count().sort_values(by=['Border','Year','Measure']).reset_index()
measure_year

In [ ]:
plt.figure(figsize=(16,10))
sns.lineplot(x='Year', y ='State',hue='Measure', data=measure_year[measure_year['Border']=='US-Mexico Border'],lw=2,
             palette ='tab10')
plt.title('Distribution of Year with respect to Measure, Border: Mexico',fontsize=18,)
plt.ylabel('Number',fontsize=14)
plt.xticks(fontsize=14)
plt.xlabel('Year',fontsize=15)
plt.yticks(fontsize=14)
plt.show()

plt.figure(figsize=(16,10))
sns.lineplot(x='Year', y ='State',hue='Measure', data=measure_year[measure_year['Border']=='US-Canada Border'],lw=2)
plt.title('Distribution of Year with respect to Measure, Border: Canada',fontsize=18,)
plt.ylabel('Number',fontsize=15)
plt.xticks(fontsize=14)
plt.xlabel('Year',fontsize=15)
plt.yticks(fontsize=14)

plt.show()

In [ ]:
year_border =df2.groupby(['Border','Year']).count().sort_values(by=['Border','Year',]).reset_index()
year_border

In [ ]:
df2[df2.Border=='US-Canada Border'].Year.value_counts()


How has the occurrence of each border changed through out the years?  

In [ ]:
plt.figure(figsize=(15,7))

sns.lineplot(data =year_border[year_border.Border== 'US-Canada Border'], x='Year',y ='State', color= 'b',lw=2)
plt.title('Yearly Change in the occurrence of US-Canada Border in the data', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylabel('Number',fontsize=15)
plt.xlabel('Year', fontsize=16)
plt.show()


plt.figure(figsize=(15,7))

sns.lineplot(data =year_border[year_border.Border== 'US-Mexico Border'], x='Year',y ='State', lw=2)
plt.title('Yearly Change in the occurrence of US-Mexico Border in the data', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylabel('Number', fontsize=15)
plt.xlabel('Year', fontsize=16)

plt.show()



From the graph above, it is shown that occurrences US-Canada Border is on 2010 while in US-Mexico Border is on 2015. 
<p> They both have it lowest occurrences in 2018-2019


How has the data changed through out the Months?  

In [ ]:
month_border =df2.groupby(['Border','Month']).count().sort_values(by=['Border','Month']).reset_index()
month_border

In [ ]:
plt.figure(figsize=(15,7))

sns.lineplot(data =month_border[month_border.Border== 'US-Canada Border'], x='Month',y ='State', color= 'b',lw=2)
plt.ylabel('Number of people')
plt.title('Monthly Change in the occurrence of US-Canada Border in the data', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylabel('Number',fontsize=15)
plt.xlabel('Month', fontsize=16)


plt.show()
plt.figure(figsize=(15,7))

sns.lineplot(data =month_border[month_border.Border== 'US-Mexico Border'], x='Month',y ='State', color='red',lw=2)
plt.ylabel('Number of people')
plt.title('Monthly Change in the occurrence of US-Mexico Border in the data', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylabel('Number',fontsize=15)
plt.xlabel('Month', fontsize=16)

plt.show()

From the graph above there is a big downwards slope towards the end of month in both Border

I am going to use Seaborn  FacetGrid to visualise it Year by Year to see how each Year affect the graph

In [ ]:
month_year =df2.groupby(['Border','Year','Month']).count().sort_values(by=['Border','Year','Month']).reset_index()
month_year

In [ ]:
# plt.subplots_adjust(top=0.5,)

g4 = sns.FacetGrid(month_year, col='Year',hue='Border', col_wrap=2, height=6)
# g4.fig.suptitle('kkk',)
g4 = (g4.map(plt.plot,'Month', 'State').add_legend())
g4.set_ylabels('Number')
# plt.subplots_adjust(top=0.5)
# g.set_title(('Monthly gross by Year'))
g4.set_xticklabels(rotation=80)
plt.show()

From the graphs above, 2019 has no entries for April to December. Also it can be shown that the big downward slope is actually only in 2016. 

What is the distribution of Value?

In [ ]:
plt.figure(figsize=(7,8))
sns.violinplot(data = df2['Value'])
plt.title('Value distribution')

We can also see that the Value column has a very long tail to right.

____

In [ ]:
value_sum =df2.groupby(['Border']).sum().sort_values(by=['Port Code']).reset_index()
value_sum

In [ ]:
plt.figure(figsize=(9,8))
ax = plt.subplot()
sns.barplot(data= value_sum, x= 'Border', y='Value',  )
plt.title('Number of people that used the borders since 1996',fontsize=18)
plt.ylabel('Value in Billion',fontsize=14)
plt.xlabel('Border',fontsize=14)

ax.set_yticks([1000000000,2000000000,3000000000,4000000000,5000000000,6000000000,7000000000])
ax.set_yticklabels(['1b','2b','3b','4b','5b','6b','7b',])
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.show()

In [ ]:
valueY_sum =df2.groupby(['Border','Year']).mean().sort_values(by=['Border','Year'],ascending=False).reset_index()
valueY_sum 

In [ ]:
plt.figure(figsize=(15,7))
ax = plt.subplot()
sns.lineplot(x='Year',y='Value',hue="Border",data=valueY_sum)

plt.title('Total Number of people that used the borders through out the years',fontsize=18)
plt.ylabel('Value in Million',fontsize=14)
ax.set_yticks([50000000,100000000,150000000,200000000,250000000,300000000,350000000,400000000])
ax.set_yticklabels(['50m','100m','150m','200m','250m','300m','350m','400m'])
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.show()

In [ ]:
plt.figure(figsize=(15,7))
ax = plt.subplot()
sns.lineplot(x='Year',y='Value',hue="Border",data=df2,)

plt.title('Average number of people that used the borders through out the years',fontsize=18)
plt.ylabel('Value in thousand ',fontsize=14)
ax.set_yticks([10000,20000,40000,60000,80000,100000,120000,140000])
ax.set_yticklabels(['10k','20k','40k','60k','80k','100k','120k','140k'])
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.show()

In [ ]:
plt.figure(figsize=(15,7))

sns.lineplot(data =df2[df2.Border== 'US-Canada Border'], x='Month',y ='Value', color= 'b',lw=2)
plt.ylabel('Number of people')
plt.title('Average monthly use, Border: Canada', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylabel('Number',fontsize=15)
plt.xlabel('Month', fontsize=16)


plt.show()
plt.figure(figsize=(15,7))

sns.lineplot(data =df2[df2.Border== 'US-Mexico Border'], x='Month',y ='Value', color='red',lw=2)
plt.ylabel('Number of people')
plt.title('Average monthly use, Border: Mexico', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylabel('Number',fontsize=15)
plt.xlabel('Month', fontsize=16)

plt.show()

In [ ]:
f =plt.figure(figsize=(16,10))

ax = plt.subplot()
plt.scatter(x='Measure',y='Value', data=((df2[df2['Border']=='US-Canada Border']).sort_values(by='Measure')),color = 'b')
plt.title('Number of people that used each Measure, Border: Canada',fontsize=18,)
plt.ylabel('Value',fontsize=14)
ax.set_yticks([100000,500000,1000000,1500000,2000000,2500000,3000000])
ax.set_yticklabels(['100k','500k','1m','1.5m','2m','2.5m','3m'])
plt.xticks(rotation=75,fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('Measure',fontsize=16)


plt.show()

plt.figure(figsize=(16,10))
ax = plt.subplot()
plt.scatter(x='Measure', y ='Value', data=(df2[df2['Border']=='US-Mexico Border'].sort_values(by='Measure')))
plt.title('Number of people that used each Measure, Border: Mexico',fontsize=18,color='black')
ax.set_yticks([100000,500000,1000000,1500000,2000000,2500000,3000000,3500000,4000000,4500000])
ax.set_yticklabels(['100k','500k','1m','1.5m','2m','2.5m','3m','3.5m','4m','4.5m'])
plt.xticks(rotation=80,fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('Measure',fontsize=15)

plt.ylabel('Value',fontsize=14)

plt.show()

In [ ]:
valuem_sum =df2.groupby(['Border','Measure']).sum().sort_values(by=['Border','Measure']).reset_index()
valuem_sum

In [ ]:
g6 = sns.FacetGrid(valuem_sum, col='Measure', col_wrap=2, size=6)
g6 = (g6.map(plt.bar,'Border', 'Value').add_legend())
g6.set_ylabels('Number of people in Hundred Millon')
g6.set_xticklabels(rotation=80)
plt.show()

In [ ]:
plt.figure(figsize=(15,7))
sns.barplot(data = df2, x= 'Measure',y= 'Value',hue ='Border')
plt.title('Average Number of people that used each Measure',fontsize=18,color='black')
plt.xticks(rotation =80)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylabel('Value',fontsize=15)
plt.xlabel('Measure', fontsize=16)
plt.show()

In [ ]:
plt.figure(figsize=(18, 6.5))
sns.scatterplot(x="State", y="Value",hue="Border", size="Border",data=df2)
plt.title('Number of people that used each State',fontsize=18,color='black')
plt.xticks(rotation =80,fontsize=14)
plt.yticks(fontsize=14)
plt.ylabel('Value',fontsize=15)
plt.xlabel('State', fontsize=16)
plt.show()

In [ ]:
plt.figure(figsize=(26, 10))
sns.scatterplot(x="Port Name", y="Value",
                hue="Border", size="Border",data=df2)
plt.title('Number of people that used each Port',fontsize=18,color='black')
plt.xticks(rotation =90)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylabel('Value',fontsize=15)
plt.xlabel('Port Name', fontsize=16)
plt.show()